Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Library Imports

In [1]:
import azureml.core
#print("SDK Version:", azureml.core.VERSION)
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

StatementMeta(spark1, 35, 1, Finished, Available)

# Read in Data from Azure Data Lake

In [2]:
data_lake_account_name = '' # Synapse Workspace ADLS
file_system_name = 'data'
table_name = "c360_data.prepareddata"

#AML workspace details
subscription_id = "" 
resource_group = "" 
workspace_name = "" 

StatementMeta(spark1, 35, 2, Finished, Available)

In [3]:
from azureml.core import Workspace
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)          

StatementMeta(spark1, 35, 3, Finished, Available)

In [4]:
sql_str = sql_str = '''SELECT cid ,pid ,uid ,InitialLeaseTerm ,(InitialLeaseTerm + ifnull((num_renewals * avg_renewal_leaseterm),0)) as TotalLeaseTerm,isRenewed,
WO_AirConditioning ,WO_AirDuctsorVents ,WO_CabinetDoors ,WO_Cabinets ,WO_Carpet ,
WO_Ceiling ,WO_CeilingFan ,WO_CleanUnit ,WO_Cleaning ,WO_Dishwasher ,WO_DoorRepairorReplace ,WO_Doorbell ,
WO_Drain ,WO_Dryer ,WO_Faucet ,WO_FilterReplacement ,WO_GarageRemote ,WO_GarbageDisposal ,WO_Heating ,WO_LightBulbs ,
WO_LightFixture ,WO_LockorExteriorDoor ,WO_LockorInteriorDoor ,WO_Microwave ,WO_Other ,WO_Outlets ,WO_Paint ,WO_PestControl ,
WO_Refrigerator ,WO_ShelvesorDrawers ,WO_ShowerHead ,WO_Sink ,WO_SinkStopper ,WO_SmokeDetector ,WO_StoveorOven ,WO_Switches ,
WO_Thermostat ,WO_Toilet ,WO_TowelBar ,WO_TrashRemoval ,WO_TuborShower ,WO_UnderSink ,WO_Walls ,WO_Washer ,WO_Windows ,
Movein_Easeofprocess ,Movein_OverallSatisfaction ,Movein_StaffCommunication ,Movein_StaffCourtesy ,Movein_StaffProfessionalism ,
Renewal_ConditionofProperty ,Renewal_MaintenanceOverallSatisfaction ,Renewal_ManagementCommunication ,Renewal_ManagementCourtesy ,
Renewal_ManagementProblemResolution ,Renewal_ManagementProfessionalism ,Renewal_OverallSatisfaction FROM c360_data.prepareddata
where isMovedOut = \'Y\'
'''
df_train = spark.sql(sql_str)

# fill null rows with 'N' for label 
df_train = df_train.na.fill(value='N', subset=['isRenewed'])

df_train.groupby('isRenewed').count().show(5)

df_train_pd = df_train.toPandas()
drop_cols = ['cid', 'pid', 'uid']
df_train_pd = df_train_pd.drop(drop_cols, axis=1)

StatementMeta(spark1, 35, 4, Finished, Available)

+---------+-----+
|isRenewed|count|
+---------+-----+
|        Y| 1218|
|        N|  594|
+---------+-----+

In [5]:
from azureml.core.experiment import Experiment
from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig

##run only once
experimentname = 'Lease-Renewal-Experiment'
experiment = Experiment(ws, experimentname)

automl_classifier_config = AutoMLConfig(
        task='classification', 
        enable_early_stopping = True,       
        experiment_timeout_minutes=15,
        iterations=2,
        primary_metric='AUC_weighted',
        training_data= df_train_pd,
        #spark_context=sc,
        #compute = 'local',
        label_column_name='isRenewed',
        n_cross_validations=5,
        model_explainability=True,
        enable_onnx_compatible_models=True,
        enable_voting_ensemble=False,
        enable_stack_ensemble=False
        )
local_run = experiment.submit(automl_classifier_config, show_output=False)

StatementMeta(spark1, 35, 5, Finished, Available)

Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'
Using default datastore for uploads

In [6]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
import joblib

# get experiment run, get the best model and register
experimentname = 'Lease-Renewal-Experiment'

for automl_run in ws.experiments[experimentname].get_runs():
    best_run, fitted_model = automl_run.get_output()  # We are taking the first run. You can update this if you like to take a different run
    break

#save the model to a local file
model_path = 'lease_renewal_model'
joblib.dump(fitted_model, model_path)

model_name = "lease_renewal_model"
registered_model = Model.register(model_path = model_path, # this points to a local file
                       model_name = model_name, # name the model is registered as
                       tags = {'type': "classification"}, 
                       description = "Lease Renewal Classifier", 
                       workspace = ws)

StatementMeta(spark1, 35, 6, Finished, Available)

Registering model lease_renewal_model